In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
#import cubic_spline_planner

N_IND_SEARCH = 10  # Search index number
show_animation = True

In [2]:
!rm -f Data_Processor.pyc

In [3]:
import numpy as np
import math
TARGET_SPEED = 1# / 3.6  # [m/s] target speed
MAX_TIME=500
T=5
NX=4
NU=2
DT=0.2
WB=50
MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None
    def array(self):
        return np.array([self.x,self.y,self.v,self.yaw],dtype=np.float64)
class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=State(x=state.x, y=state.y, yaw=state.yaw, v=state.v)
    def get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = curr_state.x + curr_state.v * math.cos(curr_state.yaw) * DT
        next_y = curr_state.y + curr_state.v * math.sin(curr_state.yaw) * DT
        next_yaw = curr_state.yaw + curr_state.v / WB * math.tan(delta) * DT
        next_v = curr_state.v + a * DT
        #print(curr_state.v,a,DT,next_v)
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)
    
    def _get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER
            
        A, B, C = get_linear_model_matrix(curr_state.v, curr_state.yaw, delta)
        
        X=np.array([curr_state.x,curr_state.y,curr_state.v,curr_state.yaw],dtype=np.float64)
        U=np.array([a,delta],dtype=np.float64)
        [next_x,next_y,next_v,next_yaw]=np.matmul(A,X) + np.matmul(B,U)+ C
        
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw),[A,B,C]


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion._get_next_state(self,accel, delta, curr_state=_state)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw
            _state=next_state
        

        return xbar


    
def get_linear_model_matrix(v, phi, delta):
        A = np.zeros((NX, NX))
        A[0, 0] = 1.0
        A[1, 1] = 1.0
        A[2, 2] = 1.0
        A[3, 3] = 1.0
        A[0, 2] = DT * math.cos(phi)
        A[0, 3] = - DT * v * math.sin(phi)#
        A[1, 2] = DT * math.sin(phi)
        A[1, 3] = DT * v * math.cos(phi)#
        A[3, 2] = DT * math.tan(delta) / WB

        B = np.zeros((NX, NU))
        B[2, 0] = DT
        B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

        C = np.zeros(NX)
        C[0] = DT * v * math.sin(phi) * phi
        C[1] = - DT * v * math.cos(phi) * phi
        C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
        #C = np.zeros(NX)
        return A, B, C
    
def np_quad_form(a,b):
        res=a @ b @ a.T
        return res
    
def calc_cost( x, u,xref,t,_B):
        
        R = np.diag([1.5, 1.5])  # input cost matrix
        Rd = np.diag([0.01, 1.0])  # input difference cost matrix
        Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
        Qf = Q  # state final matrix
        

        #cost += cvxpy.quad_form(u[:, t], R)
        #print(u,R)
        l = np_quad_form(u, R) + np_quad_form(xref - x, Q)
        
        
        #if t < (T - 1):
        #    l = np_quad_form(u[:, t + 1] - u[:, t], Rd)
            
        
        #l = np_quad_form(xref[:, T] - x[:, T], Qf)


        # compute derivatives of cost
        
        l_x1=2*Q[0,0]*(xref[0]-x[0])*(-1)
        l_x2=2*Q[1,1]*(xref[1]-x[1])*(-1)
        l_x3=2*Q[2,2]*(xref[2]-x[2])*(-1)
        l_x4=2*Q[3,3]*(xref[3]-x[3])*(-1)
            
        l_x=np.array([l_x1,l_x2,l_x3,l_x4])
        l_xx=2 * Q
        Qdiag=np.array([Q[0,0], Q[1,1], Q[2,2], Q[3,3]],dtype=np.float64)
        error=np.array(xref-x)
        #x=Ax+Bu+C
        #dx/du=d(B*u)=B
        l_u1 = 2. * u[0] * R[0,0] + np.sum(2*Qdiag*error*(-1)*_B[:,0])
        l_u2 = 2. * u[1] * R[1,1] + np.sum(2*Qdiag*error*(-1)*_B[:,1])
        
        l_u = np.array([l_u1, l_u2])

        l_uu = 2. * R
        
        l_ux = np.zeros((NU, NX))

        return l, l_x, l_xx, l_u, l_uu, l_ux

In [4]:
def sim_n_steps(_x0,U,xref,N):
    _U=U.copy()
    X=np.zeros((NX,N+1),dtype=np.float64)
    X[:,0]=_x0
    _MS=Motion()
    total_grad=0
    for t in range(N):
        _state=State(x=X[0,t],y=X[1,t],v=X[2,t],yaw=X[3,t])
        _MS.set_state(_state)
        nst,Const=_MS._get_next_state(_U[0,t],_U[1,t])
        B=Const[1]
        x_t1=[nst.x,nst.y,nst.v,nst.yaw]
        u_t=_U[:,t]
        xref_t=xref[:,t]
        cc=calc_cost(x_t1,u_t,xref_t,t,B)
        du=0.2*cc[3]
        #if t <T-1 : U[:,t+1]-=du
        X[:,t+1]=x_t1.copy()
        #print(X,cc[0],cc[3])
        total_grad+=du
    #return total_grad,U.copy()
    return X,cc,U
def sim_1_step(_X,_U):
    _MS=Motion()
     
    _state=State(x=_X[0],y=_X[1],v=_X[2],yaw=_X[3])
    _MS.set_state(_state)
    nst,Const=_MS._get_next_state(_U[0],_U[1])
    x_t1=[nst.x,nst.y,nst.v,nst.yaw]
  
    return np.array(x_t1,dtype=np.float64)

In [5]:
import numpy as np
from copy import copy, deepcopy



class iLQRController():
    def __init__(self):
        self.STATE_SIZE=NX
        self.INPUT_SIZE=NU
        self.max_iter=10
        self.dt=0.2
        self.lamb_factor=1.0
        self.lamb_max=10e4
        self.tN=T
        self.eps_converge=0.001
    def ilqr(self, x0, U,xref): 
        #U = np.zeros((NX,T),dtype=np.float64) if U is None else U
        lamb = 1.0 # regularization parameter
        sim_new_trajectory = True
        tN = U.shape[0] # number of time steps
        
        for ii in range(self.max_iter):

            if sim_new_trajectory == True: 
                # simulate forward using the current control trajectory
                X, cost, _ = sim_n_steps(x0, U,xref,T)
                oldcost = copy(cost) # copy for exit condition check

                # 
                f_x = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # df / dx
                f_u = np.zeros((self.tN, self.STATE_SIZE, self.INPUT_SIZE)) # df / du
                # for storing quadratized cost function 

                l = np.zeros((self.tN,1)) # immediate state cost 
                l_x = np.zeros((self.tN, self.STATE_SIZE)) # dl / dx
                l_xx = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # d^2 l / dx^2
                l_u = np.zeros((self.tN, self.INPUT_SIZE)) # dl / du
                l_uu = np.zeros((self.tN, self.INPUT_SIZE, self.INPUT_SIZE)) # d^2 l / du^2
                l_ux = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # d^2 l / du / dx
                # for everything except final state
                for t in range(self.tN):
                    # x(t+1) = f(x(t), u(t)) = x(t) + dx(t) * dt
                    # linearized dx(t) = np.dot(A(t), x(t)) + np.dot(B(t), u(t))
                    # f_x = (np.eye + A(t)) * dt
                    # f_u = (B(t)) * dt
                    # continuous --> discrete
                    A, B, C = get_linear_model_matrix(X[2,t],X[3,t], U[1,t])
                    f_x[t] = A
                    f_u[t] = B 

                    
                    (l[t], l_x[t], l_xx[t], l_u[t], l_uu[t], l_ux[t]) = calc_cost(X[:,t], U[:,t],xref[:,t],t,B)
                    

                # and for final state
                #l[-1], l_x[-1], l_xx[-1] = self.cost_final(X[-1])

                sim_new_trajectory = False

            V = l[-1].copy() # value function
            V_x = l_x[-1].copy() # dV / dx
            V_xx = l_xx[-1].copy() # d^2 V / dx^2
            k = np.zeros((self.tN, self.INPUT_SIZE)) # feedforward modification
            K = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # feedback gain

            # work backwards to solve for V, Q, k, and K
            for t in range(self.tN-2, -1, -1):
                
                Q_x = l_x[t] + np.dot(f_x[t].T, V_x) 
                Q_u = l_u[t] + np.dot(f_u[t].T, V_x)

                Q_xx = l_xx[t] + np.dot(f_x[t].T, np.dot(V_xx, f_x[t])) 
                Q_ux = l_ux[t] + np.dot(f_u[t].T, np.dot(V_xx, f_x[t]))
                Q_uu = l_uu[t] + np.dot(f_u[t].T, np.dot(V_xx, f_u[t]))

                Q_uu_evals, Q_uu_evecs = np.linalg.eig(Q_uu)
                Q_uu_evals[Q_uu_evals < 0] = 0.0
                Q_uu_evals += lamb
                Q_uu_inv = np.dot(Q_uu_evecs, np.dot(np.diag(1.0/Q_uu_evals), Q_uu_evecs.T))
                #k[t]=-1*l_u[t]
                k[t] = -1. * np.dot(Q_uu_inv, Q_u)
                K[t] = -1. * np.dot(Q_uu_inv, Q_ux)

                V_x = Q_x - np.dot(K[t].T, np.dot(Q_uu, k[t]))
                V_xx = Q_xx - np.dot(K[t].T, np.dot(Q_uu, K[t]))

            U_new = np.zeros((NU,T),dtype=np.float64)
            x_new = np.zeros((NX,T),dtype=np.float64)
            x_new = x0.copy()
            for t in range(self.tN - 1): 
                # use feedforward (k) and feedback (K) gain matrices 
                # calculated from our value function approximation
                U_new[:,t] = U[:,t] + k[t] + np.dot(K[t], x_new - X[:,t])
                x_new_1= sim_1_step(x_new, U_new[:,t])
                x_new=x_new_1.copy()
            # evaluate the new trajectory 
            X_new, cost_new , _= sim_n_steps(x0, U_new,xref,T)
            #if 1==1:
            if cost_new[0] < cost[0]: 
                # decrease lambda (get closer to Newton's method)
                lamb /= self.lamb_factor

                X = copy(X_new) # update trajectory 
                U = copy(U_new) # update control signal
                oldcost = copy(cost)
                cost = copy(cost_new)

                sim_new_trajectory = True # do another rollout

                # check to see if update is small enough to exit
                print('...',cost[0],oldcost[0])
                if ii > 0 and ((abs(oldcost[0]-cost[0])/cost[0]) < self.eps_converge):
                    #print("Converged at iteration = %d; Cost = %.4f;"%(ii,cost_new) + 
                    #        " logLambda = %.1f"%np.log(lamb))
                    print("break 1",ii)
                    break

            else: 
                # increase lambda (get closer to gradient descent)
                lamb *= self.lamb_factor
                # print("cost: %.4f, increasing lambda to %.4f")%(cost, lamb)
                if lamb > self.lamb_max: 
                    #print("lambda > max_lambda at iteration = %d;"%ii + 
                    #    " Cost = %.4f; logLambda = %.1f"%(cost, np.log(lamb)))
                    print("break 2",ii)
                    break

        return X, U, cost
    

In [6]:
from Car_Env import Car_Env
from Data_Processor import Data_Processor
from Car_Plot import Move_Contour,Draw_Car

In [7]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
rec=False
max_len=350
qlen=35
car_move=True
Env=Car_Env(rec=rec,max_len=max_len,qlen=qlen)
[init_state,map_points]=Env.reset()
mpx=np.array(map_points[:max_len,0])#-ofx
mpy=np.array(map_points[:max_len,1])#-ofy
ix,iy,iyaw,ivel=init_state[0]
_traj=init_state[1][0]
_dind=init_state[1][1]
_end=init_state[1][2]
ofx,ofy,ofang=init_state[2]
img=init_state[3]
Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
#print(init_state)

Track generation: 1205..1510 -> 305-tiles track


In [8]:
if rec==True:
    import os
    if not os.path.exists('Rec/'):
        os.mkdir('Rec')
    if not os.path.exists('Fig/'):
        os.mkdir('Fig')


In [9]:
import matplotlib.gridspec as gridspec
phi=0
rot_mat=np.array([[math.cos(phi),math.sin(phi)],[-math.sin(phi),math.cos(phi)]])
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]

log_state=[]
log_traj=[]
log_img=[]
log_act=[]
log_next_state=[]


#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
#cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''


_Data_Processor=Data_Processor(_speed=TARGET_SPEED,tlen=qlen)
acc_st=State(x=0,y=0)
itr=0
#fig,axes=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
fig=plt.figure(figsize=(10,10))
gs = fig.add_gridspec(3, 1)
ax1=fig.add_subplot(gs[0,:])
ax2=fig.add_subplot(gs[1:,:])
axes=[ax1,ax2]


def map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _ofang=-1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)

    _vect[0]+=_ofx
    _vect[1]+=_ofy
    return [_vect[0],_vect[1]]
def inv_map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _vect[0]-=_ofx
    _vect[1]-=_ofy
    _ofang=1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)
    return [_vect[0],_vect[1]]
U=np.zeros((NU,T),dtype=np.float64)
iLQR=iLQRController()
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        _full_state = np.array([state.x, state.y, state.v, state.yaw],dtype=np.float64)
    
        
        _full_state_traj=_Data_Processor.update_trajectory(_traj)
        xref, dref = _Data_Processor.get_ref_trajectory(state,dl,DT,T)
        
        if _end==True:
            break

        _index+=_dind
        
        #oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta)
        X,U, cost= iLQR.ilqr(_full_state, U, xref )
        oa=U[0,1]
        odelta=U[1,1]
        ox=X[0,1]
        oy=X[1,1]
        ov=X[2,1]
        oyaw=X[3,1]
        state_str=str(state.x)+','+str(state.y)+','+str(state.v)+','+str(np.rad2deg(state.yaw))+'\n'
        xref_str=str(xref[0][0])+','+str(xref[1][0])+','+str(xref[2][0])+','+str(np.rad2deg(xref[3][0]))+','+str(_index)+'\n'
        #xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str=str(accl)+','+str(gas)+','+str(np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        max_steer=np.deg2rad(45)
        max_accl=5
        if odelta is not None:
            if abs(U[1,0])<max_steer:
                  di= U[1,0]
            else:
                di=(U[1,0]/abs(U[1,0]))*max_steer
            
            if abs(U[0,0])<max_accl:
                  ai=U[0,0]
            else:
                ai=(U[0,0]/abs(U[0,0]))*max_accl
        accl=ai
        steer=di
        
        #act_str=str(steer)+'\n'
        nst,path,offset,img,rw,ter,info=Env.step([steer,accl,0])
        _traj,_dind,_end=path
        ofx,ofy,ofang=offset
        
        gas=Env.gas
        
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        Motion_Status.set_state(next_state)
        
        log_state.append(_full_state)
        log_traj.append(_full_state_traj)
        log_img.append(img)
        log_act.append([accl,steer,0])
        log_next_state.append(next_state)
        
        time = time + DT
        if car_move==True:
            _xref=xref.copy()
            [_ox,_oy]=map_plot([ox,oy],ofx,ofy,ofang)
            [_xref[0,:],_xref[1,:]]=map_plot([xref[0,:],xref[1,:]],ofx,ofy,ofang)
            [_x,_y]=map_plot([state.x,state.y],ofx,ofy,ofang)
            car_x,car_y,car_ang=ofx,ofy,ofang
            _mpx,_mpy=mpx,mpy
        else:
            _ox,_oy=ox,oy
            _xref=xref
            _x,_y=state.x,state.y
            car_x,car_y,car_ang=state.x,state.y,state.yaw
            [_mpx,_mpy]=inv_map_plot([mpx,mpy],ofx,ofy,ofang)
        x.append(_x)
        y.append(_y)
        yaw.append(ofang)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        
    
      
        
        if show_animation:  # pragma: no cover
            axes[0].cla()
            axes[1].cla()
            axes[0].text(.01,.6,'state : '+state_str,fontsize=10,color='r')
            axes[0].text(.01,.4,'xref : '+xref_str,fontsize=10,color='b')
            #axes[0].text(.01,.2,'xbar : '+xbar_str,fontsize=10,color='c')
            axes[0].text(.01,0,'act : '+act_str,fontsize=10,color='g')
            
            if ox is not None:
                axes[1].plot(_ox, _oy, "xr", label="MPC",alpha=1.0)
            axes[1].plot(_mpx, _mpy, "-b", label="course")
            axes[1].plot(x, y, "oc", label="trajectory")
            axes[1].plot(_xref[0, :], _xref[1, :], "xk", label="xref",alpha=1.0)
            axes[1].plot(_mpx[_index], _mpy[_index], "xg", label="target",alpha=1.0)
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            car_points=Draw_Car(car_x,car_y)
            rp=Move_Contour(car_points,0,0,car_ang)
            r_car_points=rp.apply()
            r_points=np.array(r_car_points,dtype=np.float64)
            axes[1].plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            #plt.title("Time[s]:" + str(round(time, 2))
            #          + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
            #          + '\nyaw'+str(state.yaw)
            #          + '\nxref_yaw'+str(xref[3][0])
            #          + '\nsteer'+str(steer))
            #fig.canvas.draw()
            plt.pause(0.0001)
            if itr%25==0:
                plt.savefig('Fig/'+str(int(itr/25))+'.jpg')
            itr+=1
if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(car_x, car_y, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


... 198.27992149934758 210.0029258546946
... 195.86935978912652 198.27992149934758
... 195.3592712466838 195.86935978912652
... 195.25059144670612 195.3592712466838
break 1 3
... 53.328282691751205 53.53451914333757
... 48.982486580243396 49.02782451301303
... 91.53229103562013 92.01781938929484
... 91.41092136753578 91.53229103562013
... 91.37872663181928 91.41092136753578
break 1 2
... 59.10235229963233 59.104066901499095
... 54.14528638019333 54.148497436303835
... 49.344099216790084 49.34538180487235
... 49.34304075631065 49.344099216790084
break 1 1
... 92.847549679083 93.61683402265524
... 92.66168499577483 92.847549679083
... 92.61378470135512 92.66168499577483
break 1 2
... 58.91408147704876 58.952167910115996
... 54.18248045406625 54.19147962414265
... 54.17961931828313 54.18248045406625
break 1 1
... 93.41488512962772 94.09744919711433
... 93.23911791490936 93.41488512962772
... 93.18882135143765 93.23911791490936
break 1 2
... 59.79858493703235 59.8056693145486
... 51.869029

In [10]:
log_dict={
'state':log_state,
'traj':log_traj,
'image':log_img,
'action':log_act,
'next_sate':log_next_state
}

In [11]:
f=open('dataset','rb')
dataset=pickle.load(f)
f.close()

NameError: name 'pickle' is not defined

In [ ]:
dataset

In [ ]:
np.arctan2(1,1)

In [ ]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [ ]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [ ]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
yaw0=np.deg2rad(90)
cyaw=[yaw-yaw0 for yaw in cyaw]

In [ ]:
import numpy as np
np.rad2deg(2.64)

In [ ]:
cyaw

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 1
n = 1
np.random.seed(1)
A = 10#np.random.randn(m, n)
b = 5#np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A*x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

In [ ]:
print(x.value)

In [ ]:
x=np.array(range(0,100))
x=x/100

In [ ]:
y=10*x-5
y=np.square(y)
plt.plot(x,y)
plt.show()

In [ ]:
import gym
gym.make('CarRacing-v0')